In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime

**Reading the CSV files**

In [4]:
cons_email_address = pd.read_csv('/content/drive/MyDrive/MissionWired/cons_email.csv')

In [5]:
cons_information = pd.read_csv('/content/drive/MyDrive/MissionWired/cons.csv')

In [6]:
cons_subs_status = pd.read_csv('/content/drive/MyDrive/MissionWired/cons_email_chapter_subscription.csv')

**Looking at the data specifications**

In [7]:
cons_information.shape, cons_information.columns

((700000, 29),
 Index(['cons_id', 'prefix', 'firstname', 'middlename', 'lastname', 'suffix',
        'salutation', 'gender', 'birth_dt', 'title', 'employer', 'occupation',
        'income', 'source', 'subsource', 'userid', 'password', 'is_validated',
        'is_banned', 'change_password_next_login', 'consent_type_id',
        'create_dt', 'create_app', 'create_user', 'modified_dt', 'modified_app',
        'modified_user', 'status', 'note'],
       dtype='object'))

In [8]:
cons_email_address.shape, cons_email_address.columns

((1400000, 16),
 Index(['cons_email_id', 'cons_id', 'cons_email_type_id', 'is_primary', 'email',
        'canonical_local_part', 'domain', 'double_validation', 'create_dt',
        'create_app', 'create_user', 'modified_dt', 'modified_app',
        'modified_user', 'status', 'note'],
       dtype='object'))

In [9]:
cons_subs_status.shape, cons_subs_status.columns

((350000, 6),
 Index(['cons_email_chapter_subscription_id', 'cons_email_id', 'chapter_id',
        'isunsub', 'unsub_dt', 'modified_dt'],
       dtype='object'))

**Considering that one person can have many emails so there is not an email address for every constituent** 

In [10]:
cons_email_address.groupby('cons_id').size().sort_values(ascending=False)

cons_id
63766     12
255248    11
606469    11
187186    11
497278    10
          ..
506674     1
271261     1
271258     1
506679     1
242530     1
Length: 605639, dtype: int64

**Similarly, cons_subs_status also have multiple rows per person** 

In [11]:
cons_subs_status.groupby('cons_email_id').size().sort_values(ascending=False)

cons_email_id
135368    6
453930    6
159290    6
272480    6
247486    6
         ..
263976    1
263977    1
263982    1
263989    1
351260    1
Length: 275484, dtype: int64

**Now looking at the subscription statuses where chapter_id = 1, all value counts are 1 and no repeats. Therefore each row represents only one instance of a person.**

In [12]:
cons_subs_status[cons_subs_status.chapter_id == 1].cons_email_id.value_counts(ascending=True)

332188    1
278185    1
514925    1
508315    1
527348    1
         ..
359698    1
373076    1
642506    1
198071    1
570       1
Name: cons_email_id, Length: 275484, dtype: int64

**Steps for merging these three files: Merge test is a trail for initial merge between cons_information and cons_email_address and filtering the entities that are required.**

In [33]:
merge_test =cons_information[['cons_id', 'create_dt', 'modified_dt']] .merge(
    cons_email_address[cons_email_address.is_primary == 1][['cons_id', 'cons_email_id', 'email', 'modified_dt']],
    on = 'cons_id',
    how = 'left'
)


**Modified_dt is a required entity. I think the modified_dt for these two files do not match. Therefore I kept the one from cons_information as required.**

In [37]:
merge_test[merge_test.modified_dt_x != merge_test.modified_dt_y]

,cons_id,create_dt,modified_dt_x,cons_email_id,email,modified_dt_y
0,1,"Fri, 1983-08-26 06:02:03","Sun, 2015-12-27 09:28:02",546912.0,daniel72@hudson.com,"Sat, 2008-05-31 02:30:35"
1,2,"Mon, 1979-03-05 21:08:54","Tue, 1989-06-20 13:28:57",415587.0,caustin@spears-carson.com,"Fri, 1982-12-17 18:55:42"
2,3,"Fri, 2008-08-22 19:20:28","Fri, 2020-06-05 18:13:57",81594.0,klewis@ford.biz,"Tue, 1992-03-24 07:15:14"
3,4,"Sun, 1984-04-29 11:18:18","Tue, 2012-01-31 07:26:35",NaN,NaN,NaN
4,5,"Thu, 2007-07-19 18:28:09","Tue, 1971-04-27 06:53:53",182741.0,stephenhamilton@gmail.com,"Sun, 1990-04-01 16:41:29"
...,...,...,...,...,...,...
699995,699996,"Mon, 2008-06-16 18:49:16","Sat, 1999-10-23 22:19:10",475997.0,cobbpatricia@anderson.biz,"Sat, 1995-04-29 23:41:43"
699996,699997,"Thu, 2001-11-15 08:04:01","Tue, 2010-08-24 07:06:01",640359.0,hjones@raymond-adkins.com,"Fri, 2009-05-01 07:31:20"
699997,699998,"Thu, 1973-06-14 08:14:54","Fri, 1995-09-29 17:28:40",434896.0,jennifer82@oneal.com,"Thu, 1984-11-15 07:53:42"
699998,699999,"Tue, 1998-06-02 01:24:00","Sun, 2014-06-01 11:16:45",786344.0,robertodixon@wilson.net,"Thu, 1974-10-24 00:31:44"


Based on the above observations: 

1.   cons_information can be joined by cons_email_address by cons_id
2.   cons_subs_status can be joined by other two using cons_email_id
3.   A given constituent can have one primary emails and many emails on file.
4.   Every row in cons_subs_status[cons_subs_status.chapter_id==1] represents one instance of one constituent. 
5.  Not all of the constituents in cons_information has corresponding email address in cons_email_address. ( that explains the NaN values). 



**Questions 1 Produce a “people” file with the following schema. Save it as a CSV with a header line to the working directory.**



**Merging the data ---- Step 1**

**Merge 1 : joining primary emails of cons_information and cons_email_address using left join as it preserves all relevant constituent level data available, also leaving missing information as NaN.**



In [38]:
merge_1 = cons_information[['cons_id', 'create_dt', 'modified_dt', 'source']].merge(
    cons_email_address[cons_email_address.is_primary == 1][['cons_id', 'email', 'cons_email_id']], on = 'cons_id', how = 'left'
)

In [39]:
merge_1.head(10)

,cons_id,create_dt,modified_dt,source,email,cons_email_id
0,1,"Fri, 1983-08-26 06:02:03","Sun, 2015-12-27 09:28:02",google,daniel72@hudson.com,546912.0
1,2,"Mon, 1979-03-05 21:08:54","Tue, 1989-06-20 13:28:57",facebook,caustin@spears-carson.com,415587.0
2,3,"Fri, 2008-08-22 19:20:28","Fri, 2020-06-05 18:13:57",NaN,klewis@ford.biz,81594.0
3,4,"Sun, 1984-04-29 11:18:18","Tue, 2012-01-31 07:26:35",google,NaN,NaN
4,5,"Thu, 2007-07-19 18:28:09","Tue, 1971-04-27 06:53:53",NaN,stephenhamilton@gmail.com,182741.0
5,6,"Sat, 2014-02-15 22:54:27","Fri, 1999-11-12 06:09:25",google,henryherring@gmail.com,246591.0
6,7,"Thu, 1979-03-01 16:12:55","Wed, 2019-03-13 02:52:10",NaN,perrymatthew@campbell.com,339289.0
7,8,"Tue, 2002-07-30 22:28:38","Sat, 1983-07-16 18:30:07",twitter,tonifranklin@yahoo.com,1057700.0
8,9,"Thu, 2019-08-29 14:50:24","Sun, 2009-12-20 14:50:14",twitter,NaN,NaN
9,10,"Tue, 1987-02-17 11:22:04","Sat, 2016-03-19 17:33:38",twitter,gillderrick@melton.net,1398350.0


**Merging cons_subs_status and merge 1**

**Merge 2 - Since we only care about subscription statuses where chapter_id =1, so I only considered that. Using right join, I merged cons_subs_status and merge 1** 

In [40]:
merge_2 = cons_subs_status[cons_subs_status.chapter_id == 1][['cons_email_id', 'isunsub']].merge(
    merge_1, on = 'cons_email_id', how = 'right'
)

In [41]:
merge_2.head(10)

,cons_email_id,isunsub,cons_id,create_dt,modified_dt,source,email
0,546912.0,NaN,1,"Fri, 1983-08-26 06:02:03","Sun, 2015-12-27 09:28:02",google,daniel72@hudson.com
1,415587.0,1.0,2,"Mon, 1979-03-05 21:08:54","Tue, 1989-06-20 13:28:57",facebook,caustin@spears-carson.com
2,81594.0,1.0,3,"Fri, 2008-08-22 19:20:28","Fri, 2020-06-05 18:13:57",NaN,klewis@ford.biz
3,NaN,NaN,4,"Sun, 1984-04-29 11:18:18","Tue, 2012-01-31 07:26:35",google,NaN
4,182741.0,1.0,5,"Thu, 2007-07-19 18:28:09","Tue, 1971-04-27 06:53:53",NaN,stephenhamilton@gmail.com
5,246591.0,NaN,6,"Sat, 2014-02-15 22:54:27","Fri, 1999-11-12 06:09:25",google,henryherring@gmail.com
6,339289.0,NaN,7,"Thu, 1979-03-01 16:12:55","Wed, 2019-03-13 02:52:10",NaN,perrymatthew@campbell.com
7,1057700.0,NaN,8,"Tue, 2002-07-30 22:28:38","Sat, 1983-07-16 18:30:07",twitter,tonifranklin@yahoo.com
8,NaN,NaN,9,"Thu, 2019-08-29 14:50:24","Sun, 2009-12-20 14:50:14",twitter,NaN
9,1398350.0,NaN,10,"Tue, 1987-02-17 11:22:04","Sat, 2016-03-19 17:33:38",twitter,gillderrick@melton.net


**Creating a "people" dataframe ----Step 2**

In [19]:
people = merge_2.drop(columns = ['cons_email_id', 'cons_id']) 
#removing the original merge fields

**Substituting isunsub NaN's with 1 -------Step 3**

In [20]:
people[people['isunsub'].isna()]
#526010 NaN's to be replaced with 1

,isunsub,create_dt,modified_dt,source,email
0,NaN,"Fri, 1983-08-26 06:02:03","Sun, 2015-12-27 09:28:02",google,daniel72@hudson.com
3,NaN,"Sun, 1984-04-29 11:18:18","Tue, 2012-01-31 07:26:35",google,NaN
5,NaN,"Sat, 2014-02-15 22:54:27","Fri, 1999-11-12 06:09:25",google,henryherring@gmail.com
6,NaN,"Thu, 1979-03-01 16:12:55","Wed, 2019-03-13 02:52:10",NaN,perrymatthew@campbell.com
7,NaN,"Tue, 2002-07-30 22:28:38","Sat, 1983-07-16 18:30:07",twitter,tonifranklin@yahoo.com
...,...,...,...,...,...
699994,NaN,"Tue, 2003-10-21 04:59:43","Thu, 2010-11-18 23:11:46",facebook,smithoscar@navarro.com
699995,NaN,"Mon, 2008-06-16 18:49:16","Sat, 1999-10-23 22:19:10",NaN,cobbpatricia@anderson.biz
699997,NaN,"Thu, 1973-06-14 08:14:54","Fri, 1995-09-29 17:28:40",twitter,jennifer82@oneal.com
699998,NaN,"Tue, 1998-06-02 01:24:00","Sun, 2014-06-01 11:16:45",google,robertodixon@wilson.net


In [21]:
people.isunsub = people.isunsub.apply(lambda x: 1 if np.isnan(x) else x).astype(bool)
#applied lambda function and assigning datatype as Boolean as required in the final data

In [22]:
people[people.isunsub== False].head(10)

,isunsub,create_dt,modified_dt,source,email
26,False,"Fri, 2016-07-29 09:28:21","Sat, 2014-01-11 11:57:25",google,hernandeztammy@hotmail.com
29,False,"Thu, 2005-11-17 16:05:17","Sat, 2013-03-23 17:57:34",facebook,kellyspears@gmail.com
153,False,"Sun, 1971-11-21 04:51:30","Mon, 1983-07-04 21:30:28",NaN,oturner@vincent.net
192,False,"Fri, 2011-08-26 18:15:06","Thu, 2009-06-18 05:56:45",NaN,jeffrey62@yahoo.com
245,False,"Wed, 1999-01-20 15:11:23","Fri, 2001-04-20 19:32:47",NaN,woodjames@wiley-wheeler.com
268,False,"Thu, 2001-03-01 19:55:43","Sat, 1994-02-12 04:40:59",NaN,aguilarlynn@yahoo.com
307,False,"Mon, 1980-09-15 12:25:12","Thu, 1978-07-20 06:32:31",organic,ewalters@benitez-bennett.com
322,False,"Fri, 1974-06-21 13:55:08","Tue, 1982-11-02 15:07:25",NaN,fgilbert@yahoo.com
323,False,"Sun, 2004-08-08 23:28:45","Sat, 1993-09-18 00:10:12",twitter,annetteburch@hotmail.com
347,False,"Mon, 2017-07-17 14:09:58","Sun, 1986-04-06 16:53:27",organic,ggarza@wallace.com


**Transform everything to correct datatypes---------Step 4**

In [23]:
people.dtypes

isunsub          bool
create_dt      object
modified_dt    object
source         object
email          object
dtype: object

In [24]:
people.create_dt,people.modified_dt = people.create_dt.apply(lambda x: pd.to_datetime(x.split(',')[1])),people.modified_dt.apply(lambda x: pd.to_datetime(x.split(',')[1]))
#changing create_dt and modified_dt datatype to actual pandas datetime format

In [25]:
people.dtypes

isunsub                  bool
create_dt      datetime64[ns]
modified_dt    datetime64[ns]
source                 object
email                  object
dtype: object

In [26]:
people['source'] = people['source'].astype('string') 
people['email']= people['email'].astype('string')
#changing datatype of source and email to string. 

In [27]:
people.dtypes

isunsub                  bool
create_dt      datetime64[ns]
modified_dt    datetime64[ns]
source                 string
email                  string
dtype: object

**Putting everything in correct format as required ------Step 5**

In [28]:
people = people[['email', 'source', 'isunsub', 'create_dt', 'modified_dt']]

**Renaming everything as required in the data ------Step 6**

In [29]:
people.rename(columns = {'source':'code','isunsub':'is_unsub','create_dt':'created_dt','modified_dt':'updated_dt'}, inplace = True)


In [30]:
people.head(10)

,email,code,is_unsub,created_dt,updated_dt
0,daniel72@hudson.com,google,True,1983-08-26 06:02:03,2015-12-27 09:28:02
1,caustin@spears-carson.com,facebook,True,1979-03-05 21:08:54,1989-06-20 13:28:57
2,klewis@ford.biz,<NA>,True,2008-08-22 19:20:28,2020-06-05 18:13:57
3,<NA>,google,True,1984-04-29 11:18:18,2012-01-31 07:26:35
4,stephenhamilton@gmail.com,<NA>,True,2007-07-19 18:28:09,1971-04-27 06:53:53
5,henryherring@gmail.com,google,True,2014-02-15 22:54:27,1999-11-12 06:09:25
6,perrymatthew@campbell.com,<NA>,True,1979-03-01 16:12:55,2019-03-13 02:52:10
7,tonifranklin@yahoo.com,twitter,True,2002-07-30 22:28:38,1983-07-16 18:30:07
8,<NA>,twitter,True,2019-08-29 14:50:24,2009-12-20 14:50:14
9,gillderrick@melton.net,twitter,True,1987-02-17 11:22:04,2016-03-19 17:33:38


**Exporting to CSV --------Step 7**

In [31]:
people.to_csv('/content/drive/MyDrive/MissionWired/people.csv', index = False)

**Question 2 Use the output of #1 to produce an “acquisition_facts” file with the following schema that aggregates stats about when people in the dataset were acquired. Save it to the working directory**

**Creating 'acquisition_facts' File**

**Extracting required information from the 'people' dataframe-----Step 1**

In [43]:
acq_facts = people[['email', 'created_dt']]

**Creating 'acquisition_date' and changing datetime to date-------Step 2**

In [44]:
acq_facts['acquisition_date'] = acq_facts['created_dt'].dt.normalize()
#because I only need date and not datetime for acquisition_date

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [45]:
acq_facts = acq_facts.drop(columns = 'created_dt') 
#dropping this column as it is not needed.

**Changing acq_facts to date and count of acquisitions on date------Step 3**

In [46]:
acq_facts = pd.DataFrame(acq_facts.acquisition_date.value_counts().reset_index().rename(
    columns = {'acquisition_date': 'acquisitions', 'index': 'acquisition_date'}
))
#putting value_counts output into a new dataframe and renaming columns 

In [47]:
acq_facts.sort_values('acquisition_date', inplace = True)

In [48]:
acq_facts.head(10)

,acquisition_date,acquisitions
13034,1970-01-01,34
8995,1970-01-02,38
15317,1970-01-03,32
8227,1970-01-04,39
3010,1970-01-05,44
5152,1970-01-06,41
9494,1970-01-07,38
14746,1970-01-08,33
6621,1970-01-09,40
7026,1970-01-10,40


**Acq_facts to CSV --------------Step 4**

In [49]:
acq_facts.to_csv('/content/drive/MyDrive/MissionWired/acquisition_facts.csv', index = False)